In [ ]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import os
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

'numpy' is imported as 'np' for numerical operations.
'VGG16' and 'preprocess_input' from Keras are used for the pre-trained VGG16 model and preprocessing images.
'image' from Keras is used for image loading and preprocessing.
'Model' from Keras is used to create a model based on VGG16.
'os' is used for file path manipulations.
'pickle' is used to load and save serialized data.
'cosine_similarity' from 'sklearn' is used to calculate the similarity between feature vectors.
'matplotlib.pyplot' is used for plotting images.

In [ ]:
# Function to extract features from an image using VGG16
def extract_features(img_path, model):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        return features.flatten()
    except UnidentifiedImageError:
        print(f"Warning: Unable to process image {img_path}")
        return None

'extract_feature's function extracts features from an image.
'img_path' is the path to the image, and 'model' is the pre-trained VGG16 model.
'image.load_img' loads the image and resizes it to 224x224 pixels.
'image.img_to_array' converts the image to a numpy array.
'np.expand_dims' adds an extra dimension to match the input shape expected by the model.
'preprocess_input' preprocesses the image array to make it suitable for the VGG16 model.
'model.predict' extracts the features from the preprocessed image.
'features.flatten' converts the features to a one-dimensional array.

In [ ]:
# Function to recommend similar dresses based on features
def recommend_similar_dresses(query_img_path, features_list, file_names, top_n=5):
    # Load pre-trained VGG16 model without the classification layer
    base_model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)
    
    # Extract features of the query image
    query_features = extract_features(query_img_path, model)
    
    # Compute cosine similarities between the query image and all dataset images
    similarities = cosine_similarity([query_features], features_list)
    
    # Get indices of top similar images
    similar_indices = similarities.argsort()[0][-top_n:][::-1]
    
    # Return top similar images
    similar_dresses = [(file_names[i], similarities[0][i]) for i in similar_indices]
    return similar_dresses

'recommend_similar_dresses' function recommends similar dresses based on the query image.
'query_img_path' is the path to the query image.
'features_list' is the list of feature vectors for the dataset images.
'file_names' is the list of image file names in the dataset.
'top_n' specifies the number of similar images to return (default is 5).
'base_model' loads the pre-trained VGG16 model without the top classification layer.
'model' creates a new model that outputs features from the 'block5_pool' layer.
'extract_features' extracts features from the query image.
'cosine_similarity' computes the cosine similarity between the query image features and the dataset features.
'similar_indices' gets the indices of the top similar images, sorted in descending order of similarity.
'similar_dresses' pairs the file names with their similarity scores for the top similar images.

In [ ]:
# Load saved features and file names
features_file = 'dress_features.pkl'
with open(features_file, 'rb') as f:
    data = pickle.load(f)
    features_list = data['features']
    file_names = data['file_names']

'features_file' is the path to the saved features file.
'pickle.load' loads the serialized data from the file.
'features_list' is assigned the list of feature vectors.
'file_names' is assigned the list of image file names.

In [ ]:
# Example usage: recommend similar dresses for a query image
query_image_path = r'C:/Users/RAMIZ/Desktop/Wahy/Notes/Project/women_fashion/Anarkali suit with a modern twist.jpg'
similar_dresses = recommend_similar_dresses(query_image_path, features_list, file_names)

'query_image_path' is the path to the query image.
'recommend_similar_dresses' is called to get the list of similar dresses.

In [ ]:
# Print recommended dresses and show the images
print('Recommended dresses:')
for dress, similarity in similar_dresses:
    print(f'Dress: {dress}, Similarity: {similarity:.2f}')

Prints the file names and similarity scores of the recommended dresses.


In [ ]:
# Display the query image and the recommended images
def display_images(query_image_path, similar_dresses, dataset_dir):
    plt.figure(figsize=(15, 10))
    
    # Show query image
    plt.subplot(2, 3, 1)
    query_img = image.load_img(query_image_path, target_size=(224, 224))
    plt.imshow(query_img)
    plt.title("Query Image")
    plt.axis('off')
    
    # Show similar images
    for i, (img_name, similarity) in enumerate(similar_dresses):
        plt.subplot(2, 3, i+2)
        img_path = os.path.join(dataset_dir, img_name)
        img = image.load_img(img_path, target_size=(224, 224))
        plt.imshow(img)
        plt.title(f"Similar: {similarity:.2f}")
        plt.axis('off')
    
    plt.show()



'display_images' function displays the query image and the recommended similar images.
'plt.figure' creates a new figure with the specified size.
'plt.subplot(2, 3, 1)' sets up the first subplot (2 rows, 3 columns, position 1) for the query image.
'image.load_img' loads the query image.
'plt.imshow' displays the query image.
'plt.title' sets the title of the query image subplot.
'plt.axis('off')' hides the axis for a cleaner display.
'for i, (img_name, similarity) in enumerate(similar_dresses)' iterates over the recommended dresses.
'plt.subplot(2, 3, i+2)' sets up the next subplot for each recommended image.
'os.path.join(dataset_dir, img_name)' constructs the full path to the recommended image.
'image.load_img' loads each recommended image.
'plt.imshow' displays each recommended image.
'plt.title' sets the title of each recommended image subplot with the similarity score.
'plt.axis('off')' hides the axis for each recommended image.
'plt.show()' displays the figure with all subplots.

In [ ]:
# Directory containing the dataset images
dataset_dir = r'C:\Users\RAMIZ\Desktop\Wahy\Notes\Project\women_fashion'  # Corrected path

display_images(query_image_path, similar_dresses, dataset_dir)

In [ ]:
'dataset_dir' specifies the directory containing the dataset images (replace with the actual path).
'display_images' is called to display the query image and the recommended images.